In [1]:
import tensorflow as tf
from tensorflow import keras
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import sys

2024-02-29 14:56:31.316296: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-02-29 14:56:37.568298: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-02-29 14:56:37.568434: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2024-02-29 14:56:37.568445: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nv

In [2]:
def create_shifted_frames_2(data):
    x = data[:, 0 : data.shape[1] - 1, :, :]
    y = data[:, data.shape[1]-1, :, :]
    return x, y

#Toma todos los colores existentes en la imagen
def get_colors(image):
  aux = []
  band = True
  for i in image:
    for j in i:

      for k in aux:
        if j.tolist() == k:
          band = False
          break
      if band:
        aux.append(j.tolist())
      band = True
  return np.array(aux)

def balance_img_categories(img, palette, balancer):
  #palette = np.sort(palette)
  rows = len(img)
  cols = len(img[0])
  for i in range(rows):
    for j in range(cols):
      pos = np.where(palette == img[i,j])[0][0]
      img[i,j] = balancer[pos]
  return img

#Función para dada una paleta solo tomar los colores de esa paleta en la imagen
def quantizetopalette(silf, palette, dither=False, mode="P"):
  """Convert an RGB or L mode image to use a given P image's palette."""
  silf.load()
  palette.load()
  im = silf.im.convert(mode, 0, palette.im)
  # the 0 above means turn OFF dithering making solid colors
  return silf._new(im)

#Realiza las operaciones necesarias para obtener una imagen RGB por una paleta de colores
def rgb_quantized(img, palette):
  rows, cols = len(img), len(img[0])
  total_vals = 1
  for i in palette.shape:
    total_vals *= i
  palettedata = palette.reshape(total_vals).tolist()
  palImage = Image.new('P', (rows, cols))
  palImage.putpalette(palettedata*32)
  oldImage = Image.fromarray(img).convert("RGB")
  newImage = quantizetopalette(oldImage,palImage)
  res_image = np.asarray(newImage.convert("RGB"))
  return res_image

def gray_quantized(img, palette):
  rows, cols = len(img), len(img[0])
  total_vals = 1
  for i in palette.shape:
    total_vals *= i
  palettedata = palette.reshape(total_vals).tolist()
  palImage = Image.new('L', (rows, cols))
  palImage.putpalette(palettedata*32)
  oldImage = Image.fromarray(img, 'L')
  newImage = quantizetopalette(oldImage,palImage, mode="L")
  res_image = np.asarray(newImage)
  return res_image

def recolor_greys_image(data, palette):
    rows, cols = len(data), len(data[0])
    aux = np.zeros((rows, cols), dtype=np.uint64)
    for i in range(rows):
        for j in range(cols):
            aux[i,j] = min(palette, key= lambda x:abs(x-data[i,j]))
    return aux

def recolor_greys_image1(data, palette):
    # Asegurarse de que la paleta y los datos estén en el mismo tipo de datos y rango
    palette = np.array(palette, dtype='float32')
    data = data.astype('float32')
    
    # Expandir las dimensiones de los datos y la paleta para la transmisión (broadcasting)
    data_expanded = data[:, :, np.newaxis]  # Forma ahora es (rows, cols, 1)
    palette_expanded = palette[np.newaxis, np.newaxis, :]  # Forma ahora es (1, 1, num_colors)
    
    # Calcular la diferencia absoluta entre cada píxel y cada color de la paleta
    abs_diff = np.abs(data_expanded - palette_expanded)
    
    # Encontrar el índice del color más cercano en la paleta para cada píxel
    indices_of_nearest = np.argmin(abs_diff, axis=2)
    
    # Mapear los índices a los valores de la paleta para obtener la imagen recoloreada
    recolored_image = palette[indices_of_nearest]
    
    return recolored_image

def agroup_window(data, window):
    new_data = [data[i:window+i] for i in range(len(data)-window+1)]
    return np.array(new_data)

def add_last(data, new_vals):
    print(f"data: {data.shape} y new_val: {new_vals.shape}")
    x_test_new = data[:,1:]
    print(f"x_test_new: {x_test_new.shape}")

    l = []
    for i in range(len(x_test_new)):
        l.append(np.append(x_test_new[i], new_vals[i]))
    x_test_new = np.array(l).reshape(data.shape[:])
    print("CX", x_test_new.shape)
    return x_test_new

def add_lastNew(data, new_val):
    print(f"data: {data.shape} y new_val: {new_val.shape}")
    x_test_new = data[:,1:,...]  # Omite el primer paso de tiempo
    print(f"x_test_new: {x_test_new.shape}")

    # Asumiendo que new_val es una única predicción que se debe añadir a cada paso de tiempo en x_test_new
    new_val = new_val.squeeze(axis=0)  # Elimina la dimensión del batch, si es necesario

    print(new_val.shape)
    # Añadir new_val a cada elemento en x_test_new
    x_test_new = np.concatenate((x_test_new, np.expand_dims(new_val, axis=1)), axis=1)

    print("CX", x_test_new.shape)
    return x_test_new

#Crea cubos con su propia información de tamaño h
def get_cubes(data, h):
    new_data = []
    for i in range(0, len(data)-h):
        new_data.append(data[i:i+h])
    new_data = np.array(new_data)
    print(new_data.shape)
    return new_data

In [3]:
channels = 1
window = 10
categories = [0, 35, 70, 119, 177, 220, 255] 
horizon = 4

parte = "EspacioLatente"

carpeta = ""

#leer una entrada de usuario por consola para variable de carpeta
carpeta = input("Ingrese el nombre de la carpeta: ")
print(carpeta)

#crear carpeta si no existe
if not os.path.exists("Resultados/ModelosConvLSTM/"+carpeta):
    os.makedirs("Resultados/ModelosConvLSTM/"+carpeta)

imagenInicial = 300

x = np.load("Resultados/NpyEspacioLatente/V1/Dataset120x360Encoded.npy")


V1


In [4]:

strategy = tf.distribute.MirroredStrategy()
#print('Number of devices: {}'.format(strategy.num_replicas_in_sync))
with strategy.scope():

    rows = x.shape[1]
    cols = x.shape[2]
    print("rows",rows)
    print("cols",cols)    
    
    print("Parte", parte)
    print("x", x.shape)
    print("x", x.dtype)
    print("x", x.min())
    print("x", x.max())

    x_2 = agroup_window(x, window)
    print(x_2.shape)
    x_train = x_2[:int(len(x_2)*.7)]
    x_test = x_2[int(len(x_2)*.7):]
    x_validation = x_train[int(len(x_train)*.8):]
    x_train = x_train[:int(len(x_train)*.8)]

    x_train = x_train.reshape(len(x_train), window, rows, cols, channels)
    x_validation = x_validation.reshape(len(x_validation), window, rows, cols, channels)
    x_test = x_test.reshape(len(x_test), window, rows, cols, channels)

    print("Forma de datos de entrenamiento: {}".format(x_train.shape))
    print("Forma de datos de validación: {}".format(x_validation.shape))
    print("Forma de datos de pruebas: {}".format(x_test.shape))

    x_train, y_train = create_shifted_frames_2(x_train)
    x_validation, y_validation = create_shifted_frames_2(x_validation)
    x_test, y_test = create_shifted_frames_2(x_test)

    print("Training dataset shapes: {}, {}".format(x_train.shape, y_train.shape))
    print("Validation dataset shapes: {}, {}".format(x_validation.shape, y_validation.shape))
    print("Test dataset shapes: {}, {}".format(x_test.shape, y_test.shape))


    np.save("Resultados/ModelosConvLSTM/"+carpeta+"/x_test_mask.npy", x_test)
    np.save("Resultados/ModelosConvLSTM/"+carpeta+"/y_test_mask.npy", y_test)
    np.save("Resultados/ModelosConvLSTM/"+carpeta+"/x_train_mask.npy", x_train)
    np.save("Resultados/ModelosConvLSTM/"+carpeta+"/y_train_mask.npy", y_train)
    np.save("Resultados/ModelosConvLSTM/"+carpeta+"/x_validation_mask.npy", x_validation)
    np.save("Resultados/ModelosConvLSTM/"+carpeta+"/y_validation_mask.npy", y_validation)

    # Define the path where you want to save the log file
    log_file_path = "Resultados/ModelosConvLSTM/"+carpeta+"/InfoConvLSTM2D_Mask"+str(rows)+"_"+str(cols)+".txt"

    # Save the original stdout so we can restore it later
    original_stdout = sys.stdout

    #Construction of Convolutional LSTM network
    inp = keras.layers.Input(shape=(None, *x_train.shape[2:]))
    #It will be constructed a 3 ConvLSTM2D layers with batch normalization,
    #Followed by a Conv3D layer for the spatiotemporal outputs.
    m = keras.layers.ConvLSTM2D(16, (5,5), padding= "same", return_sequences= True, activation= "relu")(inp)
    m = keras.layers.BatchNormalization()(m)
    m = keras.layers.ConvLSTM2D(16, (5,5), padding= "same", return_sequences= True, activation= "relu")(m)
    m = keras.layers.BatchNormalization()(m)
    m = keras.layers.ConvLSTM2D(16, (3,3), padding= "same", activation= "relu")(m)
    m = keras.layers.Conv2D(channels, (3,3), activation= "sigmoid", padding= "same")(m)
    model = keras.models.Model(inp, m)
    model.compile(loss= "binary_crossentropy", optimizer= "Adam")
    print(model.summary())
    #Callbacks
    early_stopping = keras.callbacks.EarlyStopping(monitor= "val_loss", patience= 6, restore_best_weights= True)
    reduce_lr = keras.callbacks.ReduceLROnPlateau(monitor= "val_loss", patience= 6)
    model_checkpoint = keras.callbacks.ModelCheckpoint(
        filepath= "Resultados/ModelosConvLSTM/"+carpeta+"/ConvLSTM2D_Mask"+str(rows)+"_"+str(cols)+".h5",
        monitor= "val_loss",
        save_best_only= True,
        mode= "min"
    )
    # Model training with logs redirected to a file
    with open(log_file_path, 'w') as log_file:
        sys.stdout = log_file  # Redirect stdout to the log file
        model.fit(
            x_train, y_train,
            batch_size=2,
            epochs=30,
            validation_data=(x_validation, y_validation),
            callbacks=[early_stopping, reduce_lr]
        )
        sys.stdout = original_stdout  # Restore stdout back to normal

    print(f"Training log was saved to {log_file_path}")

    #Guardar el modelo
    
    model.save("Resultados/ModelosConvLSTM/"+carpeta+"/ConvLSTM2D_Mask"+str(rows)+"_"+str(cols)+".h5")

    print(imagenInicial)

    example = x_test[imagenInicial]

    print(example.shape)

    err = model.evaluate(x_test, y_test, batch_size= 2)
    print("El error del modelo es: {}".format(err))
    preds = model.predict(x_test, batch_size= 2)
    print("preds",preds.shape)
    x_test_new = add_last(x_test, preds[:])
    preds2 = model.predict(x_test_new, batch_size= 2)
    print("preds2",preds2.shape)
    x_test_new = add_last(x_test_new, preds2[:])
    preds3 = model.predict(x_test_new, batch_size= 2)
    print ("preds3",preds3.shape)
    x_test_new = add_last(x_test_new, preds3[:])
    preds4 = model.predict(x_test_new, batch_size= 2)
    print ("preds4",preds4.shape)
    res_forecast = add_last(x_test_new, preds4[:])
    print("PREDSS",res_forecast.shape)

    np.save("Resultados/ModelosConvLSTM/"+carpeta+"/PredictionsConvolutionLSTM_forecast_"+str(rows)+"_"+str(cols)+"_"+parte+"_w"+str(window)+".npy", res_forecast)  #Guardar el vector de predicciones

    print("Res_forecast" , res_forecast.shape)

    print("x_test" , x_test.shape)
    print("x_test_new" , x_test_new.shape)
    print("y_test" , y_test.shape)

2024-02-29 14:56:52.859979: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-29 14:56:52.860493: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-29 14:56:53.316319: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-29 14:56:53.317696: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-02-29 14:56:53.319019: I tensorflow/compiler/xla/stream_executo

INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1')
rows 60
cols 180
Parte EspacioLatente
x (1254, 60, 180, 1)
x float32
x 0.19354352
x 0.9002678
(1245, 10, 60, 180, 1)
Forma de datos de entrenamiento: (696, 10, 60, 180, 1)
Forma de datos de validación: (175, 10, 60, 180, 1)
Forma de datos de pruebas: (374, 10, 60, 180, 1)
Training dataset shapes: (696, 9, 60, 180, 1), (696, 60, 180, 1)
Validation dataset shapes: (175, 9, 60, 180, 1), (175, 60, 180, 1)
Test dataset shapes: (374, 9, 60, 180, 1), (374, 60, 180, 1)
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost

2024-02-29 14:57:13.066321: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: MutableGraphView::SortTopologically error: detected edge(s) creating cycle(s) {'Func/gradient_tape/replica_1/model/conv_lstm2d_2/while/replica_1/model/conv_lstm2d_2/while_grad/body/_1679/input/_3362' -> 'gradient_tape/replica_1/model/conv_lstm2d_2/while/replica_1/model/conv_lstm2d_2/while_grad/body/_1679/gradient_tape/replica_1/model/conv_lstm2d_2/while/gradients/AddN', 'Func/gradient_tape/model/conv_lstm2d_2/while/model/conv_lstm2d_2/while_grad/body/_555/input/_2654' -> 'gradient_tape/model/conv_lstm2d_2/while/model/conv_lstm2d_2/while_grad/body/_555/gradient_tape/model/conv_lstm2d_2/while/gradients/AddN', 'Func/gradient_tape/replica_1/model/conv_lstm2d_1/while/replica_1/model/conv_lstm2d_1/while_grad/body/_1874/input/_3481' -> 'gradient_tape/replica_1/model/conv_lstm2d_1/while/replica_1/model/conv_lstm2d_1/while_grad/body/_1874/gradient_tape/replica_1/model/conv_l

Training log was saved to Resultados/ModelosConvLSTM/V1/InfoConvLSTM2D_Mask60_180.txt
300
(9, 60, 180, 1)
187/187 [==============================] - 3s 16ms/step - loss: 0.6340
El error del modelo es: 0.6340157985687256
  1/187 [..............................] - ETA: 5:39

2024-02-29 15:08:45.138857: E tensorflow/core/grappler/optimizers/meta_optimizer.cc:954] layout failed: INVALID_ARGUMENT: MutableGraphView::SortTopologically error: detected edge(s) creating cycle(s) {'replica_1/model/conv_lstm2d_2/while/body/_241/replica_1/model/conv_lstm2d_2/while/convolution_6' -> 'replica_1/model/conv_lstm2d_2/while/body/_241/replica_1/model/conv_lstm2d_2/while/add_4', 'replica_1/model/conv_lstm2d_2/while/body/_241/replica_1/model/conv_lstm2d_2/while/clip_by_value' -> 'replica_1/model/conv_lstm2d_2/while/body/_241/replica_1/model/conv_lstm2d_2/while/mul_3', 'replica_1/model/conv_lstm2d_2/while/body/_241/replica_1/model/conv_lstm2d_2/while/mul_2' -> 'replica_1/model/conv_lstm2d_2/while/body/_241/replica_1/model/conv_lstm2d_2/while/add_5', 'replica_1/model/conv_lstm2d_2/while/body/_241/replica_1/model/conv_lstm2d_2/while/clip_by_value_2' -> 'replica_1/model/conv_lstm2d_2/while/body/_241/replica_1/model/conv_lstm2d_2/while/mul_5', 'replica_1/model/conv_lstm2d_1/while/

187/187 [==============================] - 5s 16ms/step
preds (374, 60, 180, 1)
data: (374, 9, 60, 180, 1) y new_val: (374, 60, 180, 1)
x_test_new: (374, 8, 60, 180, 1)
CX (374, 9, 60, 180, 1)
187/187 [==============================] - 3s 16ms/step
preds2 (374, 60, 180, 1)
data: (374, 9, 60, 180, 1) y new_val: (374, 60, 180, 1)
x_test_new: (374, 8, 60, 180, 1)
CX (374, 9, 60, 180, 1)
187/187 [==============================] - 3s 16ms/step
preds3 (374, 60, 180, 1)
data: (374, 9, 60, 180, 1) y new_val: (374, 60, 180, 1)
x_test_new: (374, 8, 60, 180, 1)
CX (374, 9, 60, 180, 1)
187/187 [==============================] - 3s 17ms/step
preds4 (374, 60, 180, 1)
data: (374, 9, 60, 180, 1) y new_val: (374, 60, 180, 1)
x_test_new: (374, 8, 60, 180, 1)
CX (374, 9, 60, 180, 1)
PREDSS (374, 9, 60, 180, 1)
Res_forecast (374, 9, 60, 180, 1)
x_test (374, 9, 60, 180, 1)
x_test_new (374, 9, 60, 180, 1)
y_test (374, 60, 180, 1)
